# TM10007 Assignment template

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [2]:
# # Run this to use from colab environment
# !pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

In [15]:
# Data loading functions. Uncomment the one you want to use
#from adni.load_data import load_data
#from brats.load_data import load_data
from load_data import load_data
#from ecg.load_data import load_data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
import seaborn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from scipy import stats
from statsmodels.stats import weightstats

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

features = data.drop(columns=['label'])
label = data.label

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=.2)

y_train_bin = []
for val in y_train:
  if val == 'T12':
    y_train_bin.append(0)
  else:
    y_train_bin.append(1) 


The number of samples: 113
The number of columns: 160


#Preprocessing

##Removing features with certain amount of zeros

In [16]:
# def dropper(train, test, N_ZEROS):
#   X_train_drop = train.copy()
#   X_test_drop = test.copy()
#   X_dropped = []
#   for column in train.columns:
#     if list(train[column]).count(0) >N_ZEROS:
#       X_train_drop.drop(columns=column, inplace=True)
#       X_dropped.append(column)

#   X_test_drop.drop(columns=X_dropped, inplace=True)
#   print(f'Number of features before drop: {len(X_train.columns)}')
#   print(f'Number of features after drop: {len(X_train_drop.columns)}')
#   return X_train_drop, X_test_drop


# X_train_drop, X_test_drop = dropper(X_train, X_test, 10)

##Scaling

In [17]:
# Scale the dataset
scaler = RobustScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


##Feature selection

###Z-test

In [18]:
# Test normal distribution
# X_train_df = pd.DataFrame(X_train, columns = X_train.columns)
X_train['Grade'] = y_train_bin
X_T12 = X_train.groupby('Grade').get_group(0)
X_T34 = X_train.groupby('Grade').get_group(1)
X_T12 = X_T12.drop(columns = ['Grade'])
X_T34 = X_T34.drop(columns = ['Grade'])

# ztest,pval = weightstats.ztest(X_T12,X_T34)
_,pval = stats.ttest_ind(X_T12,X_T34)


print(pval.shape)
feat_sig = []
for id, val in enumerate(pval):
  if val < 0.05/len(X_train):
    feat_sig.append(list(X_train.columns)[id])

X_feat_sig_train = X_train[feat_sig]
X_feat_sig_train.columns =['Feature'+ str(pc) for pc in range(1,len(feat_sig)+1)]
# X_feat_sig_train['Grade'] = y_train_bin
#print(len(feat_sig))
# seaborn.pairplot(X_feat_sig_train, hue = 'Grade')

X_feat_sig_test = X_test[feat_sig]



(159,)


/var/folders/86/5f0w_b1966108xb6wznvlrcm0000gn/T/ipykernel_3479/1022151370.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Grade'] = y_train_bin


In [19]:

# scaler = RobustScaler()
# scaler.fit(X_feat_sig_train)
# X_train_scaled = scaler.transform(X_feat_sig_train)
# X_test_scaled = scaler.transform(X_feat_sig_test)


##PCA

In [20]:
N_COMP = 10
pca = PCA(n_components=N_COMP)
pca.fit(X_feat_sig_train)
X_train_pca = pca.transform(X_feat_sig_train)
X_test_pca = pca.transform(X_feat_sig_test)
y_train_bin = []


# seaborn.scatterplot(x=X_train_pca[:,0],y=X_train_pca[:,1],hue=y_train)
# scatter_data = pd.DataFrame(X_train_pca[:,:], columns = ['Principal component' + str(pc) for pc in range(1,N_COMP+1)])
# scatter_data['Stage'] = y_train_bin
# seaborn.pairplot(scatter_data, hue = 'Stage')
# print(scatter_data)

# Classifier

In [21]:
# kNN
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_pca, y_train)
score_train_kNN = knn.score(X_train_pca, y_train)
score_test_kNN = knn.score(X_test_pca, y_test)
print(f"Training result kNN: {score_train_kNN}")
print(f"Test result kNN: {score_test_kNN}")


# RF with PCA
# clf = RandomForestClassifier(n_estimators=5, bootstrap=True)
# clf.fit(X_train_pca, y_train)
# score_train_RF = clf.score(X_train_pca, y_train)
# score_test_RF = clf.score(X_test_pca, y_test)
# print(f"Training result Random Forest: {score_train_RF}")
# print(f"Test result Random Forest: {score_test_RF}")


# RF without PCA
clf = RandomForestClassifier(n_estimators=5, bootstrap=True)
clf.fit(X_train_scaled, y_train)
score_train_RF = clf.score(X_train_scaled, y_train)
score_test_RF = clf.score(X_test_scaled, y_test)
print(f"Training result Random Forest: {score_train_RF}")
print(f"Test result Random Forest: {score_test_RF}")

# Print result

# print(f"Test result: {score_test}")

Training result kNN: 0.7666666666666667
Test result kNN: 0.8260869565217391
Training result Random Forest: 0.9555555555555556
Test result Random Forest: 0.7391304347826086


In [ ]:
-